In [225]:
import random
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from geopandas import GeoDataFrame
import xlrd
from random import *
import requests
from io import BytesIO

pd.set_option('display.max_columns', None)

import geodata

In [226]:
gdf = gpd.read_file("../shp/SF_DOE_PREC_2022_07_18_pg.shp", encoding='utf-8')
gdf = gdf.to_crs('EPSG:4269')

import results

In [227]:
url = 'https://www.sfelections.org/results/20241105/data/20241007/psov.xlsx' 

In [232]:
response = requests.get(url)
if response.status_code == 200:
    dfTurnout = pd.read_excel(BytesIO(response.content), sheet_name='Sheet1', header=4, skipfooter=10)
    dfStateD7 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet9', header=3, skipfooter=6)
    dfStateD11 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet10', header=3, skipfooter=6)
    dfStateAd17 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet12', header=3, skipfooter=6)
    dfStateAd19 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet14', header=3, skipfooter=6)
    dfBOE = pd.read_excel(BytesIO(response.content), sheet_name='Sheet15', header=3, skipfooter=6)
    dfCC = pd.read_excel(BytesIO(response.content), sheet_name='Sheet16', header=3, skipfooter=6)
    dfBARTd7 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet17', header=3, skipfooter=6)
    dfBARTd9 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet18', header=3, skipfooter=6)
    dfMayor = pd.read_excel(BytesIO(response.content), sheet_name='Sheet19', header=3, skipfooter=6)
    dfd1 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet20', header=3, skipfooter=6)
    dfd3 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet21', header=3, skipfooter=6)
    dfd5 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet22', header=3, skipfooter=6)
    dfd7 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet23', header=3, skipfooter=6)
    dfd9 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet24', header=3, skipfooter=6)
    dfd11 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet25', header=3, skipfooter=6)
    dfCitya = pd.read_excel(BytesIO(response.content), sheet_name='Sheet26', header=3, skipfooter=6)
    dfDa = pd.read_excel(BytesIO(response.content), sheet_name='Sheet27', header=3, skipfooter=6)
    dfSheriff = pd.read_excel(BytesIO(response.content), sheet_name='Sheet28', header=3, skipfooter=6)
    dfTreasurer = pd.read_excel(BytesIO(response.content), sheet_name='Sheet29', header=3, skipfooter=6)
    dfA = pd.read_excel(BytesIO(response.content), sheet_name='Sheet40', header=3, skipfooter=6)
    dfB = pd.read_excel(BytesIO(response.content), sheet_name='Sheet41', header=3, skipfooter=6)
    dfC = pd.read_excel(BytesIO(response.content), sheet_name='Sheet42', header=3, skipfooter=6)
    dfD = pd.read_excel(BytesIO(response.content), sheet_name='Sheet43', header=3, skipfooter=6)
    dfE = pd.read_excel(BytesIO(response.content), sheet_name='Sheet44', header=3, skipfooter=6)
    dfF = pd.read_excel(BytesIO(response.content), sheet_name='Sheet45', header=3, skipfooter=6)
    dfG = pd.read_excel(BytesIO(response.content), sheet_name='Sheet46', header=3, skipfooter=6)
    dfH = pd.read_excel(BytesIO(response.content), sheet_name='Sheet47', header=3, skipfooter=6)
    dfI = pd.read_excel(BytesIO(response.content), sheet_name='Sheet48', header=3, skipfooter=6)
    dfJ = pd.read_excel(BytesIO(response.content), sheet_name='Sheet49', header=3, skipfooter=6)
    dfK = pd.read_excel(BytesIO(response.content), sheet_name='Sheet50', header=3, skipfooter=6)
    dfL = pd.read_excel(BytesIO(response.content), sheet_name='Sheet51', header=3, skipfooter=6)
    dfM = pd.read_excel(BytesIO(response.content), sheet_name='Sheet52', header=3, skipfooter=6)
    dfN = pd.read_excel(BytesIO(response.content), sheet_name='Sheet53', header=3, skipfooter=6)
    dfO = pd.read_excel(BytesIO(response.content), sheet_name='Sheet54', header=3, skipfooter=6)
else:
    print(f"Failed to download. Status code: {response.status_code}")

(takes 3 mins to run)

Shifting precinct labels down 

In [166]:
# List of DataFrame variables
# dataframes = [
#     dfTurnout, dfStateD7, dfStateD11, dfStateAd17, dfStateAd19,
#     dfBOE, dfCC, dfBARTd7, dfBARTd9, dfMayor,
#     dfd1, dfd3, dfd5, dfd7, dfd9, dfd11,
#     dfCitya, dfDa, dfSheriff, dfTreasurer,
#     dfA, dfB, dfC, dfD, dfE, dfF, dfG, dfH, dfI, dfJ, dfK, dfL, dfM, dfN, dfO
# ]

# supervisors = [
#     dfd1, dfd3, dfd5, dfd7, dfd9, dfd11
# ]

# for df in supervisors:
#     df['Precinct'] = df['Precinct'].shift(3)

In [167]:
###
### TURNOUT
###

#trim and rename columns
dfTurnout = dfTurnout[['Precinct','Registered\nVoters','Voters Cast']]
dfTurnout.columns = ['precinct','registered_voters','votes_cast']

#drop unnecessary ones
dfTurnout = dfTurnout.dropna()
dfTurnout = dfTurnout[dfTurnout["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfTurnout['precinct'] = dfTurnout['precinct'].str.replace('PCT ','').str.replace(' MB','')

#change data types
dfTurnout.registered_voters = dfTurnout.registered_voters.astype(int)
dfTurnout.votes_cast = dfTurnout.votes_cast.astype(int)

#create turnout column
dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1)
dfTurnout['turnout'] = dfTurnout['turnout'].fillna(0)

NameError: name 'dfTurnout' is not defined

In [168]:
dfd1

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,SHERMAN D'SILVA\n,Unnamed: 7,MARJAN PHILHOUR\n,Unnamed: 9,CONNIE CHAN\n,Unnamed: 11,Unnamed: 12,JEREMIAH BOEHNER\n,Unnamed: 14,JEN NOSSOKOFF\n,Unnamed: 16,Unnamed: 17,Write-in\n,Unnamed: 19,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9101,NaN,NaN,NaN,NaN,PCT 9101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,1144.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
4,Vote by Mail,1144.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Election Day,159.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
172,Vote by Mail,159.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
173,Total,159.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0
174,Electionwide - Total,46771.0,0.0,NaN,0.0,Electionwide - Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0


In [169]:
### DISTRICT 1

# make all columns lowercase and remove spaces
dfd1.columns = dfd1.columns.str.replace('\n', '')
dfd1.columns = dfd1.columns.str.lower().str.replace(' ', '_')
dfd1.columns = dfd1.columns.str.replace('_$', '')

dfd1['precinct'] = dfd1['precinct'].shift(3)

dfd1 = dfd1[['precinct', 'registered_voters', 'sherman_d\'silva', 'marjan_philhour', 'connie_chan', 'jeremiah_boehner','jen_nossokoff']]

#drop unnecessary ones
dfd1 = dfd1.dropna()
dfd1 = dfd1[dfd1["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd1['precinct'] = dfd1['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/1096422683.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd1.columns = dfd1.columns.str.replace('_$', '')


In [170]:
dfd3

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,SHARON LAI\n,Unnamed: 7,MOE JAMIL\n,Unnamed: 9,WENDY HA CHAU\n,Unnamed: 11,Unnamed: 12,EDUARD NAVARRO\n,Unnamed: 14,DANNY SAUTER\n,Unnamed: 16,Unnamed: 17,MATTHEW SUSK\n,Unnamed: 19,Write-in\n,Unnamed: 21,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 7301,NaN,NaN,NaN,NaN,PCT 7301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,1029.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0
4,Vote by Mail,1029.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Election Day,873.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0
164,Vote by Mail,873.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0
165,Total,873.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0
166,Electionwide - Total,42940.0,0.0,NaN,0.0,Electionwide - Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0


In [171]:
### DISTRICT 3

# make all columns lowercase and remove spaces
dfd3.columns = dfd3.columns.str.replace('\n', '')
dfd3.columns = dfd3.columns.str.lower().str.replace(' ', '_')
dfd3.columns = dfd3.columns.str.replace('_$', '')

dfd3['precinct'] = dfd3['precinct'].shift(3)

dfd3 = dfd3[['precinct', 'registered_voters', 'sharon_lai', 'moe_jamil', 'wendy_ha_chau', 'eduard_navarro','danny_sauter', 'matthew_susk']]

#drop unnecessary ones
dfd3 = dfd3.dropna()
dfd3 = dfd3[dfd3["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd3['precinct'] = dfd3['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/1285040468.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd3.columns = dfd3.columns.str.replace('_$', '')


In [172]:
dfd5.head(1)

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,AUTUMN HOPE LOOIJEN\n,Unnamed: 7,BILAL MAHMOOD\n,Unnamed: 9,SCOTTY JACOBS\n,Unnamed: 11,Unnamed: 12,ALLEN JONES\n,Unnamed: 14,DEAN PRESTON\n,Unnamed: 16,Unnamed: 17,Write-in\n,Unnamed: 19,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
### DISTRICT 5

# make all columns lowercase and remove spaces
dfd5.columns = dfd5.columns.str.replace('\n', '')
dfd5.columns = dfd5.columns.str.lower().str.replace(' ', '_')
dfd5.columns = dfd5.columns.str.replace('_$', '')

dfd5['precinct'] = dfd5['precinct'].shift(3)

dfd5 = dfd5[['precinct', 'registered_voters', 'autumn_hope_looijen', 'bilal_mahmood', 'scotty_jacobs', 'allen_jones','dean_preston']]

#drop unnecessary ones
dfd5 = dfd5.dropna()
dfd5 = dfd5[dfd5["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd5['precinct'] = dfd5['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/2909828452.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd5.columns = dfd5.columns.str.replace('_$', '')


In [174]:
dfd7.head(1)

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,MYRNA MELGAR\n,Unnamed: 7,STEPHEN MARTIN-PINTO\n,Unnamed: 9,EDWARD S. YEE\n,Unnamed: 11,Unnamed: 12,MATT BOSCHETTO\n,Unnamed: 14,Write-in\n,Unnamed: 16,Unnamed: 17,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
### DISTRICT 7

# make all columns lowercase and remove spaces
dfd7.columns = dfd7.columns.str.replace('\n', '')
dfd7.columns = dfd7.columns.str.replace('.', '')
dfd7.columns = dfd7.columns.str.lower().str.replace(' ', '_')
dfd7.columns = dfd7.columns.str.replace('_$', '')

dfd7['precinct'] = dfd7['precinct'].shift(3)

dfd7 = dfd7[['precinct', 'registered_voters', 'myrna_melgar', 'stephen_martin-pinto', 'edward_s_yee', 'matt_boschetto']]

#drop unnecessary ones
dfd7 = dfd7.dropna()
dfd7 = dfd7[dfd7["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd7['precinct'] = dfd7['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/616033664.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfd7.columns = dfd7.columns.str.replace('.', '')
/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/616033664.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd7.columns = dfd7.columns.str.replace('_$', '')


In [176]:
dfd9.columns

Index(['Precinct', 'Registered \nVoters', 'Undervotes', 'Unnamed: 3',
       'Overvotes', 'Precinct.1', 'JACKIE FIELDER\n ', 'Unnamed: 7',
       'STEPHEN JON TORRES\n ', 'Unnamed: 9', 'ROBERTO HERNANDEZ\n ',
       'Unnamed: 11', 'Unnamed: 12', 'JAIME GUTIERREZ\n ', 'Unnamed: 14',
       'TREVOR CHANDLER\n ', 'Unnamed: 16', 'Unnamed: 17',
       'JULIAN BERMUDEZ\n ', 'Unnamed: 19', 'H. BROWN\n ', 'Unnamed: 21',
       'Write-in\n ', 'Unnamed: 23', 'Total Votes'],
      dtype='object')

In [177]:
### DISTRICT 9

# make all columns lowercase and remove spaces
dfd9.columns = dfd9.columns.str.replace('\n', '')
dfd9.columns = dfd9.columns.str.replace('.', '')
dfd9.columns = dfd9.columns.str.lower().str.replace(' ', '_')
dfd9.columns = dfd9.columns.str.replace('_$', '')

dfd9['precinct'] = dfd9['precinct'].shift(3)

dfd9 = dfd9[['precinct', 'registered_voters', 'jackie_fielder', 'stephen_jon_torres', 'roberto_hernandez', 'jaime_gutierrez','trevor_chandler','julian_bermudez', 'h_brown']]

#drop unnecessary ones
dfd9 = dfd9.dropna()
dfd9 = dfd9[dfd9["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd9['precinct'] = dfd9['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/457179141.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfd9.columns = dfd9.columns.str.replace('.', '')
/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/457179141.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd9.columns = dfd9.columns.str.replace('_$', '')


In [178]:
dfd11.head()

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,OSCAR FLORES\n,Unnamed: 7,MICHAEL LAI\n,Unnamed: 9,ROGER K. MARENCO\n,Unnamed: 11,Unnamed: 12,JOSE MORALES\n,Unnamed: 14,"ERNEST ""EJ"" JONES\n",Unnamed: 16,Unnamed: 17,ADLAH CHISTI\n,Unnamed: 19,CHYANNE CHEN\n,Unnamed: 21,Write-in\n,Unnamed: 23,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 1101,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,887.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,Vote by Mail,887.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [179]:
### DISTRICT 11

# make all columns lowercase and remove spaces
dfd11.columns = dfd11.columns.str.replace('\n', '')
dfd11.columns = dfd11.columns.str.replace('"', '')
dfd11.columns = dfd11.columns.str.replace('.', '')
dfd11.columns = dfd11.columns.str.lower().str.replace(' ', '_')
dfd11.columns = dfd11.columns.str.replace('_$', '')

dfd11['precinct'] = dfd11['precinct'].shift(3)

dfd11 = dfd11[['precinct', 'registered_voters', 'oscar_flores', 'michael_lai', 'roger_k_marenco', 'jose_morales','ernest_ej_jones','adlah_chisti', 'chyanne_chen']]

#drop unnecessary ones
dfd11 = dfd11.dropna()
dfd11 = dfd11[dfd11["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd11['precinct'] = dfd11['precinct'].str.replace('PCT ','').str.replace(' MB','')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/3974194942.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dfd11.columns = dfd11.columns.str.replace('.', '')
/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/3974194942.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dfd11.columns = dfd11.columns.str.replace('_$', '')


In [263]:
dfMayor = pd.read_excel(BytesIO(response.content), sheet_name='Sheet19', header=3, skipfooter=6)

In [264]:
dfMayor.columns = dfMayor.columns.str.replace('\n', '')
dfMayor.columns = dfMayor.columns.str.lower().str.replace(' ', '_')
dfMayor.columns = dfMayor.columns.str.replace('_$', '')

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/2162369028.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dfMayor.columns = dfMayor.columns.str.replace('_$', '')


In [265]:
dfMayor.head(20)

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,henry_flynn,unnamed:_7,mark_farrell,unnamed:_9,keith_freedman,unnamed:_11,unnamed:_12,daniel_lurie,unnamed:_14,dylan_hirsch-shell,unnamed:_16,unnamed:_17,nelson_mei,unnamed:_19,paul_ybarra_robertson,unnamed:_21,aaron_peskin,unnamed:_23,london_breed,unnamed:_25,shahram_shariati,unnamed:_27,ahsha_safaí_,unnamed:_29,jon_soderstrom,unnamed:_31,ellen_lee_zhou,unnamed:_33,write-in,unnamed:_35,total_votes,michael_linqualified_write_in,unnamed:_38,marc_rothqualified_write_in
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 1101,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,887.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
4,Vote by Mail,887.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
5,Total,887.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
6,PCT 1102,NaN,NaN,NaN,NaN,PCT 1102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Election Day,898.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
8,Vote by Mail,898.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
9,Total,898.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0


In [266]:
dfMayor['precinct'] = dfMayor['precinct'].shift(3)

dfMayor.head(20)

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,henry_flynn,unnamed:_7,mark_farrell,unnamed:_9,keith_freedman,unnamed:_11,unnamed:_12,daniel_lurie,unnamed:_14,dylan_hirsch-shell,unnamed:_16,unnamed:_17,nelson_mei,unnamed:_19,paul_ybarra_robertson,unnamed:_21,aaron_peskin,unnamed:_23,london_breed,unnamed:_25,shahram_shariati,unnamed:_27,ahsha_safaí_,unnamed:_29,jon_soderstrom,unnamed:_31,ellen_lee_zhou,unnamed:_33,write-in,unnamed:_35,total_votes,michael_linqualified_write_in,unnamed:_38,marc_rothqualified_write_in
0,None,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,PCT 1101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Countywide,887.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
4,Electionwide,887.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
5,PCT 1101,887.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
6,Election Day,NaN,NaN,NaN,NaN,PCT 1102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Vote by Mail,898.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
8,Total,898.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0
9,PCT 1102,898.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,0.0


In [268]:
dfMayor = dfMayor.dropna()


In [269]:
dfMayor.head(0)

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,henry_flynn,unnamed:_7,mark_farrell,unnamed:_9,keith_freedman,unnamed:_11,unnamed:_12,daniel_lurie,unnamed:_14,dylan_hirsch-shell,unnamed:_16,unnamed:_17,nelson_mei,unnamed:_19,paul_ybarra_robertson,unnamed:_21,aaron_peskin,unnamed:_23,london_breed,unnamed:_25,shahram_shariati,unnamed:_27,ahsha_safaí_,unnamed:_29,jon_soderstrom,unnamed:_31,ellen_lee_zhou,unnamed:_33,write-in,unnamed:_35,total_votes,michael_linqualified_write_in,unnamed:_38,marc_rothqualified_write_in


In [245]:
dfMayor = dfMayor.dropna()


dfMayor

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,henry_flynn,unnamed:_7,mark_farrell,unnamed:_9,keith_freedman,unnamed:_11,unnamed:_12,daniel_lurie,unnamed:_14,dylan_hirsch-shell,unnamed:_16,unnamed:_17,nelson_mei,unnamed:_19,paul_ybarra_robertson,unnamed:_21,aaron_peskin,unnamed:_23,london_breed,unnamed:_25,shahram_shariati,unnamed:_27,ahsha_safaí_,unnamed:_29,jon_soderstrom,unnamed:_31,ellen_lee_zhou,unnamed:_33,write-in,unnamed:_35,total_votes,michael_linqualified_write_in,unnamed:_38,marc_rothqualified_write_in


In [250]:
### MAYOR

dfMayor.columns = dfMayor.columns.str.replace('\n', '')
dfMayor.columns = dfMayor.columns.str.lower().str.replace(' ', '_')
dfMayor.columns = dfMayor.columns.str.replace('_$', '')

dfMayor['precinct'] = dfMayor['precinct'].shift(3)

dfMayor = dfMayor[['precinct', 'registered_voters', 'henry_flynn', 'mark_farrell', 'keith_freedman', 'daniel_lurie', 'dylan_hirsh-shell', 'nelson_mei', 'paul_ybarra_robertson', 'aaron_peskin', 'london_breed', 'shahram_shariati', 'ahsha_sahfaí', 'jon_soderstrom', 'ellen_lee_zhou']]

dfMayor = dfMayor.dropna()
dfMayor['precinct'] = dfMayor[dfMayor['precinct'].str.contains('PCT')]

dfMayor['precinct'] = dfMayor['precinct'].str.replace('PCT ','').str.replace(' MB','')

dfMayor

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_83102/991126443.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfMayor.columns = dfMayor.columns.str.replace('_$', '')


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [254]:
dfMayor['precinct'] = dfMayor[dfMayor['precinct'].str.contains('PCT')]

dfMayor['precinct'] = dfMayor['precinct'].str.replace('PCT ','').str.replace(' MB','')

dfMayor

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [239]:
dfMayor

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,henry_flynn,unnamed:_7,mark_farrell,unnamed:_9,keith_freedman,unnamed:_11,unnamed:_12,daniel_lurie,unnamed:_14,dylan_hirsch-shell,unnamed:_16,unnamed:_17,nelson_mei,unnamed:_19,paul_ybarra_robertson,unnamed:_21,aaron_peskin,unnamed:_23,london_breed,unnamed:_25,shahram_shariati,unnamed:_27,ahsha_safaí_,unnamed:_29,jon_soderstrom,unnamed:_31,ellen_lee_zhou,unnamed:_33,write-in,unnamed:_35,total_votes,michael_linqualified_write_in,unnamed:_38,marc_rothqualified_write_in


REMOVE LATER !!!

In [180]:
# randmoize votes in candidate columns

for index, row in dfd1.iterrows():
    dfd1.at[index, 'sherman_d\'silva'] = randint(0, 1000)
    dfd1.at[index, 'marjan_philhour'] = randint(0, 1000)
    dfd1.at[index, 'connie_chan'] = randint(0, 1000)
    dfd1.at[index, 'jeremiah_boehner'] = randint(0, 1000)
    dfd1.at[index, 'jen_nossokoff'] = randint(0, 1000)


# do the same for district 3

for index, row in dfd3.iterrows():
    dfd3.at[index, 'sharon_lai'] = randint(0, 1000)
    dfd3.at[index, 'moe_jamil'] = randint(0, 1000)
    dfd3.at[index, 'wendy_ha_chau'] = randint(0, 1000)
    dfd3.at[index, 'eduard_navarro'] = randint(0, 1000)
    dfd3.at[index, 'danny_sauter'] = randint(0, 1000)
    dfd3.at[index, 'matthew_susk'] = randint(0, 1000)

# do the same for district 5

for index, row in dfd5.iterrows():
    dfd5.at[index, 'autumn_hope_looijen'] = randint(0, 1000)
    dfd5.at[index, 'bilal_mahmood'] = randint(0, 1000)
    dfd5.at[index, 'scotty_jacobs'] = randint(0, 1000)
    dfd5.at[index, 'allen_jones'] = randint(0, 1000)
    dfd5.at[index, 'dean_preston'] = randint(0, 1000)

# do the same for district 7

for index, row in dfd7.iterrows():
    dfd7.at[index, 'myrna_melgar'] = randint(0, 1000)
    dfd7.at[index, 'stephen_martin-pinto'] = randint(0, 1000)
    dfd7.at[index, 'edward_s_yee'] = randint(0, 1000)
    dfd7.at[index, 'matt_boschetto'] = randint(0, 1000)

# do the same for district 9

for index, row in dfd9.iterrows():
    dfd9.at[index, 'jackie_fielder'] = randint(0, 1000)
    dfd9.at[index, 'stephen_jon_torres'] = randint(0, 1000)
    dfd9.at[index, 'roberto_hernandez'] = randint(0, 1000)
    dfd9.at[index, 'jaime_gutierrez'] = randint(0, 1000)
    dfd9.at[index, 'trevor_chandler'] = randint(0, 1000)
    dfd9.at[index, 'julian_bermudez'] = randint(0, 1000)
    dfd9.at[index, 'h_brown'] = randint(0, 1000)

# do the same for district 11

for index, row in dfd11.iterrows():
    dfd11.at[index, 'oscar_flores'] = randint(0, 1000)
    dfd11.at[index, 'michael_lai'] = randint(0, 1000)
    dfd11.at[index, 'roger_k_marenco'] = randint(0, 1000)
    dfd11.at[index, 'jose_morales'] = randint(0, 1000)
    dfd11.at[index, 'ernest_ej_jones'] = randint(0, 1000)
    dfd11.at[index, 'adlah_chisti'] = randint(0, 1000)
    dfd11.at[index, 'chyanne_chen'] = randint(0, 1000)


In [181]:
dfd1['total_votes'] = dfd1['sherman_d\'silva'] + dfd1['marjan_philhour'] + dfd1['connie_chan'] + dfd1['jeremiah_boehner'] + dfd1['jen_nossokoff']

dfd3['total_votes'] = dfd3['sharon_lai'] + dfd3['moe_jamil'] + dfd3['wendy_ha_chau'] + dfd3['eduard_navarro'] + dfd3['danny_sauter'] + dfd3['matthew_susk']

dfd5['total_votes'] = dfd5['autumn_hope_looijen'] + dfd5['bilal_mahmood'] + dfd5['scotty_jacobs'] + dfd5['allen_jones'] + dfd5['dean_preston']

dfd7['total_votes'] = dfd7['myrna_melgar'] + dfd7['stephen_martin-pinto'] + dfd7['edward_s_yee'] + dfd7['matt_boschetto']

dfd9['total_votes'] = dfd9['jackie_fielder'] + dfd9['stephen_jon_torres'] + dfd9['roberto_hernandez'] + dfd9['jaime_gutierrez'] + dfd9['trevor_chandler'] + dfd9['julian_bermudez'] + dfd9['h_brown']

dfd11['total_votes'] = dfd11['oscar_flores'] + dfd11['michael_lai'] + dfd11['roger_k_marenco'] + dfd11['jose_morales'] + dfd11['ernest_ej_jones'] + dfd11['adlah_chisti'] + dfd11['chyanne_chen']

In [182]:
# calculate a percentage of votes for each candidate AS a new column
dfd1['sherman_d\'silva_p'] = round((dfd1['sherman_d\'silva'] / dfd1['total_votes']) * 100, 1)
dfd1['marjan_philhour_p'] = round((dfd1['marjan_philhour'] / dfd1['total_votes']) * 100, 1)
dfd1['connie_chan_p'] = round((dfd1['connie_chan'] / dfd1['total_votes']) * 100, 1)
dfd1['jeremiah_boehner_p'] = round((dfd1['jeremiah_boehner'] / dfd1['total_votes']) * 100, 1)
dfd1['jen_nossokoff_p'] = round((dfd1['jen_nossokoff'] / dfd1['total_votes']) * 100, 1)

dfd3['sharon_lai_p'] = round((dfd3['sharon_lai'] / dfd3['total_votes']) * 100, 1)
dfd3['moe_jamil_p'] = round((dfd3['moe_jamil'] / dfd3['total_votes']) * 100, 1)
dfd3['wendy_ha_chau_p'] = round((dfd3['wendy_ha_chau'] / dfd3['total_votes']) * 100, 1)
dfd3['eduard_navarro_p'] = round((dfd3['eduard_navarro'] / dfd3['total_votes']) * 100, 1)
dfd3['danny_sauter_p'] = round((dfd3['danny_sauter'] / dfd3['total_votes']) * 100, 1)
dfd3['matthew_susk_p'] = round((dfd3['matthew_susk'] / dfd3['total_votes']) * 100, 1)

dfd5['autumn_hope_looijen_p'] = round((dfd5['autumn_hope_looijen'] / dfd5['total_votes']) * 100, 1)
dfd5['bilal_mahmood_p'] = round((dfd5['bilal_mahmood'] / dfd5['total_votes']) * 100, 1)
dfd5['scotty_jacobs_p'] = round((dfd5['scotty_jacobs'] / dfd5['total_votes']) * 100, 1)
dfd5['allen_jones_p'] = round((dfd5['allen_jones'] / dfd5['total_votes']) * 100, 1)
dfd5['dean_preston_p'] = round((dfd5['dean_preston'] / dfd5['total_votes']) * 100, 1)

dfd7['myrna_melgar_p'] = round((dfd7['myrna_melgar'] / dfd7['total_votes']) * 100, 1)
dfd7['stephen_martin-pinto_p'] = round((dfd7['stephen_martin-pinto'] / dfd7['total_votes']) * 100, 1)
dfd7['edward_s_yee_p'] = round((dfd7['edward_s_yee'] / dfd7['total_votes']) * 100, 1)
dfd7['matt_boschetto_p'] = round((dfd7['matt_boschetto'] / dfd7['total_votes']) * 100, 1)

dfd9['jackie_fielder_p'] = round((dfd9['jackie_fielder'] / dfd9['total_votes']) * 100, 1)
dfd9['stephen_jon_torres_p'] = round((dfd9['stephen_jon_torres'] / dfd9['total_votes']) * 100, 1)
dfd9['roberto_hernandez_p'] = round((dfd9['roberto_hernandez'] / dfd9['total_votes']) * 100, 1)
dfd9['jaime_gutierrez_p'] = round((dfd9['jaime_gutierrez'] / dfd9['total_votes']) * 100, 1)
dfd9['trevor_chandler_p'] = round((dfd9['trevor_chandler'] / dfd9['total_votes']) * 100, 1)
dfd9['julian_bermudez_p'] = round((dfd9['julian_bermudez'] / dfd9['total_votes']) * 100, 1)
dfd9['h_brown_p'] = round((dfd9['h_brown'] / dfd9['total_votes']) * 100, 1)

dfd11['oscar_flores_p'] = round((dfd11['oscar_flores'] / dfd11['total_votes']) * 100, 1)
dfd11['michael_lai_p'] = round((dfd11['michael_lai'] / dfd11['total_votes']) * 100, 1)
dfd11['roger_k_marenco_p'] = round((dfd11['roger_k_marenco'] / dfd11['total_votes']) * 100, 1)
dfd11['jose_morales_p'] = round((dfd11['jose_morales'] / dfd11['total_votes']) * 100, 1)
dfd11['ernest_ej_jones_p'] = round((dfd11['ernest_ej_jones'] / dfd11['total_votes']) * 100, 1)
dfd11['adlah_chisti_p'] = round((dfd11['adlah_chisti'] / dfd11['total_votes']) * 100, 1)
dfd11['chyanne_chen_p'] = round((dfd11['chyanne_chen'] / dfd11['total_votes']) * 100, 1)

In [183]:
# make a column that shows the candidate with the highest percentage of votes in each precinct

dfd1['winner'] = dfd1[['sherman_d\'silva_p', 'marjan_philhour_p', 'connie_chan_p', 'jeremiah_boehner_p', 'jen_nossokoff_p']].idxmax(axis=1).str.replace('_p', '')
dfd3['winner'] = dfd3[['sharon_lai_p', 'moe_jamil_p', 'wendy_ha_chau_p', 'eduard_navarro_p', 'danny_sauter_p', 'matthew_susk_p']].idxmax(axis=1).str.replace('_p', '')
dfd5['winner'] = dfd5[['autumn_hope_looijen_p', 'bilal_mahmood_p', 'scotty_jacobs_p', 'allen_jones_p', 'dean_preston_p']].idxmax(axis=1).str.replace('_p', '')
dfd7['winner'] = dfd7[['myrna_melgar_p', 'stephen_martin-pinto_p', 'edward_s_yee_p', 'matt_boschetto_p']].idxmax(axis=1).str.replace('_p', '')
dfd9['winner'] = dfd9[['jackie_fielder_p', 'stephen_jon_torres_p', 'roberto_hernandez_p', 'jaime_gutierrez_p', 'trevor_chandler_p', 'julian_bermudez_p', 'h_brown_p']].idxmax(axis=1).str.replace('_p', '')
dfd11['winner'] = dfd11[['oscar_flores_p', 'michael_lai_p', 'roger_k_marenco_p', 'jose_morales_p', 'ernest_ej_jones_p', 'adlah_chisti_p', 'chyanne_chen_p']].idxmax(axis=1).str.replace('_p', '')

In [217]:
# in the winner column in dfd1 replace marjanhilhour with marjan_philhour

dfd1['winner'] = dfd1['winner'].replace('marjanhilhour', 'marjan_philhour')
dfd5['winner'] = dfd5['winner'].replace('deanreston', 'dean_preston')

In [186]:
bins = [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 100]
labels = ['Less than 25%', '25-30%', '30-35%', '35-40%', '40-45%', '45-50%', '50-55%', '55-60%', '60-65%', '65-70%', '70-75%', '75% and more']

# Calculate voter turnout percentage
dfTurnout['yes_perc'] = (dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100

# Categorize turnout percentages into bins
dfTurnout['yes_perc'] = pd.cut(dfTurnout['yes_perc'], bins=bins, labels=labels, include_lowest=True)


NameError: name 'dfTurnout' is not defined

In [187]:
dfTurnout

NameError: name 'dfTurnout' is not defined

In [224]:
dfd9.winner.value_counts()

julian_bermudez       10
jackie_fielder         8
jaime_gutierrez        8
h_brown                7
roberto_hernandez      6
trevor_chandler        5
stephen_jon_torres     2
Name: winner, dtype: int64

## merge

In [188]:
#tidy up shapefile
gdf = gdf[['Prec_2022','geometry']]
gdf.Prec_2022 = gdf.Prec_2022.astype(str)

In [189]:
dfTurnout = gdf.merge(dfTurnout, right_on='precinct', left_on='Prec_2022')
dfTurnout = dfTurnout[['precinct','registered_voters','votes_cast','yes_perc','turnout','geometry']]
gdfTurnout = GeoDataFrame(dfTurnout, crs="EPSG:4269", geometry='geometry')

NameError: name 'dfTurnout' is not defined

In [190]:
gdf

,Prec_2022,geometry
0,7042,"POLYGON ((-122.42165 37.71029, -122.42100 37.7..."
1,1107,"POLYGON ((-122.45595 37.71134, -122.45593 37.7..."
2,1145,"POLYGON ((-122.44617 37.71104, -122.44550 37.7..."
3,7043,"POLYGON ((-122.41508 37.71166, -122.41289 37.7..."
4,7046,"POLYGON ((-122.40973 37.71195, -122.40883 37.7..."
...,...,...
509,9201,"POLYGON ((-122.47754 37.81011, -122.47750 37.8..."
510,9202,"POLYGON ((-122.47725 37.81102, -122.47711 37.8..."
511,7301,"POLYGON ((-122.41991 37.81163, -122.41990 37.8..."
512,7645,"POLYGON ((-122.36862 37.83116, -122.36733 37.8..."


In [191]:
dfd1 = gdf.merge(dfd1, right_on='precinct', left_on='Prec_2022')

In [192]:
dfd3 = gdf.merge(dfd3, right_on='precinct', left_on='Prec_2022')

In [218]:
dfd5 = gdf.merge(dfd5, right_on='precinct', left_on='Prec_2022')

In [194]:
dfd7 = gdf.merge(dfd7, right_on='precinct', left_on='Prec_2022')

In [195]:
dfd9 = gdf.merge(dfd9, right_on='precinct', left_on='Prec_2022')

In [196]:
dfd11 = gdf.merge(dfd11, right_on='precinct', left_on='Prec_2022')

In [197]:
dfd1.head()

,Prec_2022,geometry,precinct,registered_voters,sherman_d'silva,marjan_philhour,connie_chan,jeremiah_boehner,jen_nossokoff,total_votes,sherman_d'silva_p,marjan_philhour_p,connie_chan_p,jeremiah_boehner_p,jen_nossokoff_p,winner
0,9147,"POLYGON ((-122.45126 37.77470, -122.45116 37.7...",9147,159.0,885.0,699.0,20.0,779.0,401.0,2784.0,31.8,25.1,0.7,28.0,14.4,sherman_d'silva
1,9133,"POLYGON ((-122.50682 37.77341, -122.50669 37.7...",9133,1110.0,849.0,689.0,58.0,448.0,561.0,2605.0,32.6,26.4,2.2,17.2,21.5,sherman_d'silva
2,9134,"POLYGON ((-122.50146 37.77365, -122.50133 37.7...",9134,1255.0,125.0,477.0,623.0,772.0,315.0,2312.0,5.4,20.6,26.9,33.4,13.6,jeremiah_boehner
3,9135,"POLYGON ((-122.49612 37.77390, -122.49597 37.7...",9135,1216.0,243.0,18.0,452.0,854.0,284.0,1851.0,13.1,1.0,24.4,46.1,15.3,jeremiah_boehner
4,9136,"POLYGON ((-122.49075 37.77414, -122.49062 37.7...",9136,1200.0,912.0,899.0,824.0,805.0,854.0,4294.0,21.2,20.9,19.2,18.7,19.9,sherman_d'silva


In [213]:
# save to geojson

dfd1.to_file("../docs/supes-map/data/1.geojson", driver='GeoJSON')

In [199]:
dfd3.to_file("../docs/supes-map/data/3.geojson", driver='GeoJSON')

In [219]:
dfd5.to_file("../docs/supes-map/data/5.geojson", driver='GeoJSON')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [201]:
dfd7.to_file("../docs/supes-map/data/7.geojson", driver='GeoJSON')

In [202]:
dfd9.to_file("../docs/supes-map/data/9.geojson", driver='GeoJSON')

In [203]:
dfd11.to_file("../docs/supes-map/data/11.geojson", driver='GeoJSON')

In [140]:
# save to geojson
# Convert the 'yes_perc' column to string type for compatibility with GeoJSON export
dfTurnout['yes_perc'] = dfTurnout['yes_perc'].astype(str)

# Now you can export to GeoJSON
gdfTurnout.to_file('../docs/turnout-map/data/turnout.geojson', driver='GeoJSON')
